In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import df_from_csv, to_be_checked, is_corr_ok, plot_hist, plot_boxes, plot_lines, df_to_csv, drop_pm25_outliers

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## PM2.5 Clean Data

In [2]:
# Load Raw Data
df_pm25_raw = df_from_csv("pm25_raw.csv")

# Check Data Types
print(df_pm25_raw.dtypes)

df_pm25_raw.head()

TANGARA_260A    float64
TANGARA_4B1A    float64
TANGARA_14D6    float64
TANGARA_2B42    float64
TANGARA_2E9A    float64
TANGARA_2FF6    float64
TANGARA_307A    float64
TANGARA_48C6    float64
TANGARA_F1AE    float64
TANGARA_06BE    float64
dtype: object


,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,
2023-11-09 00:00:00-05:00,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,9.0,NaN
2023-11-09 00:00:30-05:00,11.0,12.0,NaN,9.0,3.0,11.0,NaN,3.0,10.0,12.0
2023-11-09 00:01:00-05:00,12.0,12.0,10.0,9.0,2.0,13.0,NaN,6.0,10.0,14.0
2023-11-09 00:01:30-05:00,12.0,12.0,10.0,9.0,3.0,13.0,NaN,6.0,10.0,14.0
2023-11-09 00:02:00-05:00,11.0,13.0,10.0,10.0,2.0,13.0,NaN,9.0,10.0,14.0


## Descriptive Statistics

In [3]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5609.000000,5639.000000,3836.000000,4181.000000,5666.000000,5683.000000,3226.000000,5506.000000,5723.000000,5522.000000
mean,9.393475,11.711296,12.083942,7.288448,6.432227,7.373394,7.687539,6.244279,9.574699,9.730170
std,4.733606,7.959368,6.522791,4.396990,8.252764,3.117201,2.824535,3.828438,4.974644,4.887244
min,1.000000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,6.000000,8.000000,8.000000,3.000000,3.000000,5.000000,6.000000,4.000000,6.000000,6.000000
50%,9.000000,9.000000,11.000000,8.000000,6.000000,8.000000,8.000000,6.000000,8.000000,10.000000
75%,12.000000,12.000000,14.000000,10.000000,9.000000,9.000000,9.000000,8.000000,12.000000,12.000000
max,27.000000,76.000000,53.000000,23.000000,527.000000,17.000000,43.000000,27.000000,73.000000,94.000000


## Missing Data

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_pm25_raw.columns:
    df_pm25_sensor = df_pm25_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_pm25_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_14D6, Data: 67%, Missing: 33%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 73%, Missing: 27%, To be checked
Tangara Sensor: TANGARA_307A, Data: 56%, Missing: 44%, To be checked


## Data Correlation

In [5]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_pm25_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_pm25_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_pm25_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_pm25_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")

Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


## Histograms

In [6]:
# Plot Histograms
plot_hist(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Boxplots

In [7]:
# Plot Boxplots
plot_boxes(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Lineplots

In [8]:
# Plot Lineplots
plot_lines(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Drop PM2.5 Outliers

In [9]:
# Drop PM2.5 Outliers
df_pm25_clean, resume = drop_pm25_outliers(df_pm25_raw)

In [10]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5490.000000,5037.000000,3651.000000,4175.000000,5550.000000,5679.000000,3142.000000,5266.000000,5616.000000,5434.000000
mean,9.083060,9.471511,11.036154,7.267784,6.025946,7.367142,7.475812,5.734333,9.244302,9.391976
std,4.279498,3.087780,4.265543,4.366055,3.948631,3.109358,2.447387,3.013592,4.259290,3.870799
min,1.000000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,6.000000,7.000000,8.000000,3.000000,3.000000,5.000000,6.000000,4.000000,6.000000,6.000000
50%,9.000000,9.000000,11.000000,8.000000,6.000000,8.000000,8.000000,6.000000,8.000000,10.000000
75%,12.000000,11.000000,13.000000,10.000000,9.000000,9.000000,9.000000,7.000000,11.000000,12.000000
max,21.000000,18.000000,23.000000,20.000000,18.000000,15.000000,13.000000,14.000000,21.000000,21.000000


In [11]:
# Describe Data
df_pm25_clean.describe()

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5490.000000,5037.000000,3651.000000,4175.000000,5550.000000,5679.000000,3142.000000,5266.000000,5616.000000,5434.000000
mean,9.083060,9.471511,11.036154,7.267784,6.025946,7.367142,7.475812,5.734333,9.244302,9.391976
std,4.279498,3.087780,4.265543,4.366055,3.948631,3.109358,2.447387,3.013592,4.259290,3.870799
min,1.000000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,6.000000,7.000000,8.000000,3.000000,3.000000,5.000000,6.000000,4.000000,6.000000,6.000000
50%,9.000000,9.000000,11.000000,8.000000,6.000000,8.000000,8.000000,6.000000,8.000000,10.000000
75%,12.000000,11.000000,13.000000,10.000000,9.000000,9.000000,9.000000,7.000000,11.000000,12.000000
max,21.000000,18.000000,23.000000,20.000000,18.000000,15.000000,13.000000,14.000000,21.000000,21.000000


## Histograms

In [12]:
# Plot Histograms
plot_hist(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Boxplots

In [13]:
# Plot Boxplots
plot_boxes(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Lineplots

In [14]:
# Plot Lineplots
plot_lines(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


In [15]:
# Save PM2.5 Data Frame Sensors into CSV file
df_to_csv(df_pm25_clean, "pm25_clean.csv", datafolder='1_clean')